In [2]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# 결측값 처리
train.fillna('WT', inplace=True)
test.fillna('WT', inplace=True)

# 변이 열 준비
mutation_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]

# 변이 정보를 문자열로 결합
train['mutations'] = train[mutation_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1)
test['mutations'] = test[mutation_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1)

# 타겟 레이블 인코딩
label_encoder = LabelEncoder()
train['SUBCLASS'] = label_encoder.fit_transform(train['SUBCLASS'])
num_classes = len(label_encoder.classes_)

# 특징과 타겟 준비
X = train['mutations']
y = train['SUBCLASS']

# 텍스트 데이터를 숫자 특징으로 변환하기 위해 TF-IDF 벡터화를 사용
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X_tfidf = tfidf.fit_transform(X)
X_test_tfidf = tfidf.transform(test['mutations'])

# 학습 데이터를 훈련 세트와 검증 세트로 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)

# 희소 행렬을 밀집 배열로 변환
X_train_dense = X_train.toarray()
X_valid_dense = X_valid.toarray()
X_test_dense = X_test_tfidf.toarray()

In [ ]:
# TPOT 임포트
from tpot import TPOTClassifier
from sklearn.metrics import classification_report

# TPOT 분류기 생성
tpot = TPOTClassifier(
    generations=5,  # 파이프라인 최적화 반복 횟수
    population_size=50,
    verbosity=2,
    n_jobs=-1,
    max_time_mins=60,  # 총 시간 제한 (분)
    scoring='neg_log_loss'
)

# TPOT 모델 학습
tpot.fit(X_train_dense, y_train)

# 검증 데이터에 대한 예측
y_pred = tpot.predict(X_valid)

# 성능 평가
print("TPOT Classification Report:")
print(classification_report(y_valid_dense, y_pred, target_names=label_encoder.classes_))

# 테스트 데이터에 대한 예측
y_test_pred = tpot.predict(X_test_dense)
test['SUBCLASS'] = label_encoder.inverse_transform(y_test_pred)

# 제출 파일 생성
submission = test[['ID', 'SUBCLASS']]
submission.to_csv('submission_tpot.csv', index=False)
print("TPOT 제출 파일이 생성되었습니다: submission_tpot.csv")

# 최적의 파이프라인 코드 내보내기
tpot.export('tpot_best_pipeline.py')


/opt/conda/envs/tpot4/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


Optimization Progress:   0%|          | 0/50 [00:00<?, ?pipeline/s]

/opt/conda/envs/tpot4/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/opt/conda/envs/tpot4/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/opt/conda/envs/tpot4/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/opt/conda/envs/tpot4/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not a